In [1]:
#Import desired tools needed for data collections and analysis
import numpy as np
import pandas as pd
import json
import requests
import gmaps
import matplotlib.pyplot as plt
from scipy.stats import linregress
from us import states
from config import api_key, gmaps_api_key
from census import Census
from config import census_key
c = Census(census_key, year=2018)

In [3]:
#Read josn file listing all cities in the OpenWeather API
city_list = pd.read_json("city.list.json", orient="list")

#Select US cities from list
US_city_list = city_list.loc[city_list["country"] == "US", :]
US_city_list

,id,name,state,country,coord
141897,4046255,Bay Minette,AL,US,"{'lon': -87.773048, 'lat': 30.882959}"
141898,4046274,Edna,TX,US,"{'lon': -96.646088, 'lat': 28.97859}"
141899,4046319,Bayou La Batre,AL,US,"{'lon': -88.24852, 'lat': 30.403521}"
141900,4046332,Henderson,TX,US,"{'lon': -94.799377, 'lat': 32.153221}"
141901,4046430,Natalia,TX,US,"{'lon': -98.862534, 'lat': 29.18968}"
...,...,...,...,...,...
208905,8122840,Western Community Park,KS,US,"{'lon': -97.680717, 'lat': 38.367432}"
208906,8122866,Happy Homes Park,KS,US,"{'lon': -99.976273, 'lat': 37.7458}"
208907,8123060,Western Mobile Home Park South,KS,US,"{'lon': -100.939072, 'lat': 37.03735}"
209568,8260172,West Greenwich,RI,US,"{'lon': -71.660042, 'lat': 41.637001}"


In [4]:
#Separate coordinates into 2 data series for longitude and latitude
US_Cities = pd.concat([US_city_list.drop(['coord'], axis=1), US_city_list['coord'].apply(pd.Series)], axis=1)
US_Cities

,id,name,state,country,lon,lat
141897,4046255,Bay Minette,AL,US,-87.773048,30.882959
141898,4046274,Edna,TX,US,-96.646088,28.978590
141899,4046319,Bayou La Batre,AL,US,-88.248520,30.403521
141900,4046332,Henderson,TX,US,-94.799377,32.153221
141901,4046430,Natalia,TX,US,-98.862534,29.189680
...,...,...,...,...,...,...
208905,8122840,Western Community Park,KS,US,-97.680717,38.367432
208906,8122866,Happy Homes Park,KS,US,-99.976273,37.745800
208907,8123060,Western Mobile Home Park South,KS,US,-100.939072,37.037350
209568,8260172,West Greenwich,RI,US,-71.660042,41.637001


In [5]:
#Select only cities in Kansas
KS_city_list = US_Cities.loc[US_Cities["state"] == "KS", :].reset_index()
del KS_city_list["index"]
KS_city_list

,id,name,state,country,lon,lat
0,4048488,Stockton,KS,US,-99.265099,39.438068
1,4267354,Abilene,KS,US,-97.213913,38.917221
2,4267387,Adamsville,KS,US,-97.176430,37.173359
3,4267427,Akron,KS,US,-97.015038,37.351688
4,4267513,Alma,KS,US,-96.289162,39.016670
...,...,...,...,...,...,...
297,8122749,Gaslight Mobile Home Park,KS,US,-94.942368,39.756969
298,8122819,Southwestern Mobile Home Park,KS,US,-97.620148,38.802349
299,8122840,Western Community Park,KS,US,-97.680717,38.367432
300,8122866,Happy Homes Park,KS,US,-99.976273,37.745800


In [6]:
#Create data frame with columns for desired climate information
KS_city_data = pd.DataFrame({"City": KS_city_list["name"],
                             "Lat": KS_city_list["lat"],
                             "Lng": KS_city_list["lon"],
                             "Timezone_Offset": "",
                             "Current Date": "",
                             "Current Temp": "",
                             "Current Humidity": "",
                             "Day1 Date": "",
                             "Day1 Temp": "",
                             "Day1 Low": "",
                             "Day1 High": "",
                             "Day1 Humidity": "",
                             "Day1 Weather": "",
                             "Day2 Date": "",
                             "Day2 Temp": "",
                             "Day2 Low": "",
                             "Day2 High": "",
                             "Day2 Humidity": "",
                             "Day2 Weather": "",
                             "Day3 Date": "",
                             "Day3 Temp": "",
                             "Day3 Low": "",
                             "Day3 High": "",
                             "Day3 Humidity": "",
                             "Day3 Weather": "",
                             "Day4 Date": "",
                             "Day4 Temp": "",
                             "Day4 Low": "",
                             "Day4 High": "",
                             "Day4 Humidity": "",
                             "Day4 Weather": "",
                             "Day5 Date": "",
                             "Day5 Temp": "",
                             "Day5 Low": "",
                             "Day5 High": "",
                             "Day5 Humidity": "",
                             "Day5 Weather": "",
                             })
KS_city_data

,City,Lat,Lng,Timezone_Offset,Current Date,Current Temp,Current Humidity,Day1 Date,Day1 Temp,Day1 Low,...,Day4 Low,Day4 High,Day4 Humidity,Day4 Weather,Day5 Date,Day5 Temp,Day5 Low,Day5 High,Day5 Humidity,Day5 Weather
0,Stockton,39.438068,-99.265099,,,,,,,,...,,,,,,,,,,
1,Abilene,38.917221,-97.213913,,,,,,,,...,,,,,,,,,,
2,Adamsville,37.173359,-97.176430,,,,,,,,...,,,,,,,,,,
3,Akron,37.351688,-97.015038,,,,,,,,...,,,,,,,,,,
4,Alma,39.016670,-96.289162,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,Gaslight Mobile Home Park,39.756969,-94.942368,,,,,,,,...,,,,,,,,,,
298,Southwestern Mobile Home Park,38.802349,-97.620148,,,,,,,,...,,,,,,,,,,
299,Western Community Park,38.367432,-97.680717,,,,,,,,...,,,,,,,,,,
300,Happy Homes Park,37.745800,-99.976273,,,,,,,,...,,,,,,,,,,


In [7]:
#Query the OpenWeather API and place desired data into data frame

part = "minutely,hourly"

for index, row in KS_city_list.iterrows():
    lat = row["lat"]
    lon = row["lon"]

    URL = f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={part}&units=imperial&appid={api_key}"
    response = requests.get(URL).json()
    
    KS_city_data.loc[index,"Timezone_Offset"] = response["timezone_offset"]
    KS_city_data.loc[index,"Current Date"] = response["current"]["dt"]
    KS_city_data.loc[index,"Current Temp"] = response["current"]["temp"]
    KS_city_data.loc[index,"Current Humidity"] = response["current"]["humidity"]
    KS_city_data.loc[index,"Day1 Date"] = response["daily"][0]["dt"]
    KS_city_data.loc[index,"Day1 Temp"] = response["daily"][0]["temp"]["day"]
    KS_city_data.loc[index,"Day1 Low"] = response["daily"][0]["temp"]["min"]
    KS_city_data.loc[index,"Day1 High"] = response["daily"][0]["temp"]["max"]
    KS_city_data.loc[index,"Day1 Humidity"] = response["daily"][0]["humidity"]
    KS_city_data.loc[index,"Day1 Weather"] = response["daily"][0]["weather"][0]["main"]
    KS_city_data.loc[index,"Day2 Date"] = response["daily"][1]["dt"]
    KS_city_data.loc[index,"Day2 Temp"] = response["daily"][1]["temp"]["day"]
    KS_city_data.loc[index,"Day2 Low"] = response["daily"][1]["temp"]["min"]
    KS_city_data.loc[index,"Day2 High"] = response["daily"][1]["temp"]["max"]
    KS_city_data.loc[index,"Day2 Humidity"] = response["daily"][1]["humidity"]
    KS_city_data.loc[index,"Day2 Weather"] = response["daily"][1]["weather"][0]["main"]
    KS_city_data.loc[index,"Day3 Date"] = response["daily"][2]["dt"]
    KS_city_data.loc[index,"Day3 Temp"] = response["daily"][2]["temp"]["day"]
    KS_city_data.loc[index,"Day3 Low"] = response["daily"][2]["temp"]["min"]
    KS_city_data.loc[index,"Day3 High"] = response["daily"][2]["temp"]["max"]
    KS_city_data.loc[index,"Day3 Humidity"] = response["daily"][2]["humidity"]
    KS_city_data.loc[index,"Day3 Weather"] = response["daily"][2]["weather"][0]["main"]
    KS_city_data.loc[index,"Day4 Date"] = response["daily"][3]["dt"]
    KS_city_data.loc[index,"Day4 Temp"] = response["daily"][3]["temp"]["day"]
    KS_city_data.loc[index,"Day4 Low"] = response["daily"][3]["temp"]["min"]
    KS_city_data.loc[index,"Day4 High"] = response["daily"][3]["temp"]["max"]
    KS_city_data.loc[index,"Day4 Humidity"] = response["daily"][3]["humidity"]
    KS_city_data.loc[index,"Day4 Weather"] = response["daily"][3]["weather"][0]["main"]
    KS_city_data.loc[index,"Day5 Date"] = response["daily"][4]["dt"]
    KS_city_data.loc[index,"Day5 Temp"] = response["daily"][4]["temp"]["day"]
    KS_city_data.loc[index,"Day5 Low"] = response["daily"][4]["temp"]["min"]
    KS_city_data.loc[index,"Day5 High"] = response["daily"][4]["temp"]["max"]
    KS_city_data.loc[index,"Day5 Humidity"] = response["daily"][4]["humidity"]
    KS_city_data.loc[index,"Day5 Weather"] = response["daily"][4]["weather"][0]["main"]
    
KS_city_data

KeyboardInterrupt: 

In [ ]:
# Configure gmaps
gmaps.configure(api_key=gmaps_api_key)

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [ ]:
#define coordinates from cities_df
coordinates = []
temperatures = []

for index, row in KS_city_data.iterrows():
    lat = row["Lat"]
    lon = row["Lng"]
    coordinates.append((lat, lon))
    
    temp = row["Day1 High"]
    temperatures.append(str(temp))

coordinates

In [ ]:
# Map temperature in all cities in list
fig = gmaps.figure(center=(38.5,-98.35), zoom_level=7)

heat_layer = gmaps.heatmap_layer(coordinates, weights=KS_city_data["Current Temp"], 
                                 dissipating=False, max_intensity=KS_city_data["Current Temp"].max(),
                                 point_radius = 0.1)

# Add the layer to the map
fig.add_layer(heat_layer)
fig

In [ ]:
#Narrow list to cities withing +/- 0.5 deg latitude and -0.4 degress longitude
KC_area = KS_city_data.loc[((KS_city_data["Lat"] > 38.59)&(KS_city_data["Lat"] < 39.59)&(KS_city_data["Lng"] > -94.98))]
KC_area

In [ ]:
US_Cities

In [ ]:
MO_Cities = US_Cities.loc[US_Cities["state"] == "MO", :].reset_index()
del MO_Cities["index"]
MO_Cities

In [ ]:
KC_MO_area = MO_Cities.loc[((MO_Cities["lat"] > 38.59)&(MO_Cities["lat"] < 39.59)
                            &(MO_Cities["lon"] < -94.18))].reset_index()
del KC_MO_area["index"]
KC_MO_area

In [ ]:
KC_MO_data = pd.DataFrame({"City": KC_MO_area["name"],
                             "Lat": KC_MO_area["lat"],
                             "Lng": KC_MO_area["lon"],
                             "Timezone_Offset": "",
                             "Current Date": "",
                             "Current Temp": "",
                             "Current Humidity": "",
                             "Day1 Date": "",
                             "Day1 Temp": "",
                             "Day1 Low": "",
                             "Day1 High": "",
                             "Day1 Humidity": "",
                             "Day1 Weather": "",
                             "Day2 Date": "",
                             "Day2 Temp": "",
                             "Day2 Low": "",
                             "Day2 High": "",
                             "Day2 Humidity": "",
                             "Day2 Weather": "",
                             "Day3 Date": "",
                             "Day3 Temp": "",
                             "Day3 Low": "",
                             "Day3 High": "",
                             "Day3 Humidity": "",
                             "Day3 Weather": "",
                             "Day4 Date": "",
                             "Day4 Temp": "",
                             "Day4 Low": "",
                             "Day4 High": "",
                             "Day4 Humidity": "",
                             "Day4 Weather": "",
                             "Day5 Date": "",
                             "Day5 Temp": "",
                             "Day5 Low": "",
                             "Day5 High": "",
                             "Day5 Humidity": "",
                             "Day5 Weather": "",
                             })
KC_MO_data

In [ ]:
#Query the OpenWeather API and place desired data into data frame

part = "minutely,hourly"

for index, row in KC_MO_data.iterrows():
    lat = row["Lat"]
    lon = row["Lng"]

    URL = f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude={part}&units=imperial&appid={api_key}"
    response = requests.get(URL).json()
    
    KC_MO_data.loc[index,"Timezone_Offset"] = response["timezone_offset"]
    KC_MO_data.loc[index,"Current Date"] = response["current"]["dt"]
    KC_MO_data.loc[index,"Current Temp"] = response["current"]["temp"]
    KC_MO_data.loc[index,"Current Humidity"] = response["current"]["humidity"]
    KC_MO_data.loc[index,"Day1 Date"] = response["daily"][0]["dt"]
    KC_MO_data.loc[index,"Day1 Temp"] = response["daily"][0]["temp"]["day"]
    KC_MO_data.loc[index,"Day1 Low"] = response["daily"][0]["temp"]["min"]
    KC_MO_data.loc[index,"Day1 High"] = response["daily"][0]["temp"]["max"]
    KC_MO_data.loc[index,"Day1 Humidity"] = response["daily"][0]["humidity"]
    KC_MO_data.loc[index,"Day1 Weather"] = response["daily"][0]["weather"][0]["main"]
    KC_MO_data.loc[index,"Day2 Date"] = response["daily"][1]["dt"]
    KC_MO_data.loc[index,"Day2 Temp"] = response["daily"][1]["temp"]["day"]
    KC_MO_data.loc[index,"Day2 Low"] = response["daily"][1]["temp"]["min"]
    KC_MO_data.loc[index,"Day2 High"] = response["daily"][1]["temp"]["max"]
    KC_MO_data.loc[index,"Day2 Humidity"] = response["daily"][1]["humidity"]
    KC_MO_data.loc[index,"Day2 Weather"] = response["daily"][1]["weather"][0]["main"]
    KC_MO_data.loc[index,"Day3 Date"] = response["daily"][2]["dt"]
    KC_MO_data.loc[index,"Day3 Temp"] = response["daily"][2]["temp"]["day"]
    KC_MO_data.loc[index,"Day3 Low"] = response["daily"][2]["temp"]["min"]
    KC_MO_data.loc[index,"Day3 High"] = response["daily"][2]["temp"]["max"]
    KC_MO_data.loc[index,"Day3 Humidity"] = response["daily"][2]["humidity"]
    KC_MO_data.loc[index,"Day3 Weather"] = response["daily"][2]["weather"][0]["main"]
    KC_MO_data.loc[index,"Day4 Date"] = response["daily"][3]["dt"]
    KC_MO_data.loc[index,"Day4 Temp"] = response["daily"][3]["temp"]["day"]
    KC_MO_data.loc[index,"Day4 Low"] = response["daily"][3]["temp"]["min"]
    KC_MO_data.loc[index,"Day4 High"] = response["daily"][3]["temp"]["max"]
    KC_MO_data.loc[index,"Day4 Humidity"] = response["daily"][3]["humidity"]
    KC_MO_data.loc[index,"Day4 Weather"] = response["daily"][3]["weather"][0]["main"]
    KC_MO_data.loc[index,"Day5 Date"] = response["daily"][4]["dt"]
    KC_MO_data.loc[index,"Day5 Temp"] = response["daily"][4]["temp"]["day"]
    KC_MO_data.loc[index,"Day5 Low"] = response["daily"][4]["temp"]["min"]
    KC_MO_data.loc[index,"Day5 High"] = response["daily"][4]["temp"]["max"]
    KC_MO_data.loc[index,"Day5 Humidity"] = response["daily"][4]["humidity"]
    KC_MO_data.loc[index,"Day5 Weather"] = response["daily"][4]["weather"][0]["main"]
    
KC_MO_data

In [ ]:
#Add KC MO data to KC area data - this data frame represents the KC Metro area
KC_Metro = KC_area.append(KC_MO_data, ignore_index=True)

In [ ]:
KC_Metro[["Day1 High", "Day2 High", "Day3 High"]

In [ ]:
KC_coord = []
KC_markers = []

for index, row in KC_Metro.iterrows():
    lat = row["Lat"]
    lon = row["Lng"]
    KC_coord.append((lat, lon))


#for index, row in KC_Metro.iterrows():
    temp = row["Day1 High"]
    KC_markers.append(str(temp))
KC_markers

In [ ]:
fig1 = gmaps.figure(layout=figure_layout)
fig1 = gmaps.figure(center=(39.1,-94.58), zoom_level=9.1)

heat_layer = gmaps.heatmap_layer(KC_coord, weights=KC_Metro["Day1 High"], 
                                 dissipating=False, max_intensity=KC_Metro["Day1 High"].max(),
                                 point_radius = .1)

marker = gmaps.marker_layer(KC_coord, label=KC_markers)

# Add the layer to the map
fig1.add_layer(heat_layer)

fig1.add_layer(marker)


fig1

In [ ]:
#Find the lat/lon coordinates for KC MO
KC_MO_lat = KC_MO_data["Lat"].loc[KC_MO_data["City"] == "Kansas City"]
KC_MO_lon = KC_MO_data["Lng"].loc[KC_MO_data["City"] == "Kansas City"]
print(KC_MO_lat)
#locate KC_MO row in the KC_Metro dataframe

KC_Metro.loc[KC_Metro["City"] == "Kansas City"]
#KC MO is row 43 in KC_Metro dataframe

In [ ]:



#Find KC MO temps on Days 1 - 5
KC_MO_D1_T = KC_MO_data["Day1 High"].loc[KC_MO_data["City"] == "Kansas City"]
KC_MO_D2_T = KC_MO_data["Day2 High"].loc[KC_MO_data["City"] == "Kansas City"]
KC_MO_D3_T = KC_MO_data["Day3 High"].loc[KC_MO_data["City"] == "Kansas City"]
KC_MO_D4_T = KC_MO_data["Day4 High"].loc[KC_MO_data["City"] == "Kansas City"]
KC_MO_D5_T = KC_MO_data["Day5 High"].loc[KC_MO_data["City"] == "Kansas City"]

In [ ]:
KC_Metro.columns

In [ ]:
#Create new data frame to compare distance from KC MO to temperature
KC_Metro_comparison = KC_Metro[["City", "Lat", "Lng", "Day1 High", 
                               "Day2 High", "Day3 High", "Day4 High",
                               "Day5 High"]]
#KC_Metro_comparison = KC_Metro_comparison(columns=["Chng Lat", "Chng Lng", 
#                                                            "Day1 Chng", "Day2 Chng", 
#                                                            "Day3 Chng", "Day4 Chng", 
#                                                            "Day5 Chng"])

for index, row in KC_Metro_comparison.iterrows():

    KC_Metro_comparison.loc[index, "Chng Lat"] = abs(KC_Metro.iloc[43, 1] - row["Lat"])
    KC_Metro_comparison.loc[index, "Chng Lng"] = abs(KC_Metro.iloc[43, 2] - row["Lng"])
    KC_Metro_comparison.loc[index, "Day1 Chng"] = row["Day1 High"] - KC_Metro.iloc[43, 10]
    KC_Metro_comparison.loc[index, "Day2 Chng"] = row["Day2 High"] - KC_Metro.iloc[43, 16]
    KC_Metro_comparison.loc[index, "Day3 Chng"] = row["Day3 High"] - KC_Metro.iloc[43, 22]
    KC_Metro_comparison.loc[index, "Day4 Chng"] = row["Day4 High"] - KC_Metro.iloc[43, 28]
    KC_Metro_comparison.loc[index, "Day5 Chng"] = row["Day5 High"] - KC_Metro.iloc[43, 34]


KC_Metro_comparison

In [ ]:
#Calculating distance from changes in latitude and longitude 
#formulas from https://www.sisense.com/blog/latitude-longitude-distance-calculation-explained/

#a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)

#c = 2 * atan2( √a, √(1−a) )

#d = R * c

#Where φ represent the latitudes, and λ represent the longitudes.
KClat = KC_Metro.iloc[43, 1]

for index, row in KC_Metro_comparison.iterrows():
    KC_Metro_comparison.loc[index, "a"] = np.sin(np.deg2rad(row["Chng Lat"])/2)**2 + ((np.cos(np.deg2rad(row["Lat"]))) * (np.cos(np.deg2rad(KClat))) * (np.sin(np.deg2rad(row["Chng Lng"])/2)**2))    
for index, row in KC_Metro_comparison.iterrows():    
    KC_Metro_comparison.loc[index, "Distance (miles)"] = round((2 * (np.arctan2((np.sqrt(row["a"])), (np.sqrt(1-row["a"]))))) *3958.8, 2)
    
#for index, row in KC_Metro_comparison.iterrows():  
#    KC_Metro_comparison.loc[index, "Distance"] = 3,958.8 * row["c"]
#distance = 3,958.8 * c

KC_Metro_comparison

In [ ]:
#Troubleshooting calcs - discovered numpy calculates radians by default and lat/lon is measured in degrees
#convert all var from degrees to radians for the above calculations
lat = np.deg2rad(39.141670)
lon = np.deg2rad(-94.883583)
chlat = np.deg2rad(0.041939)
chlon = np.deg2rad(0.360015)
KClat = np.deg2rad(KC_Metro.iloc[43, 1])
np.sin(chlat/2)**2 + ((np.cos(lat)) * (np.cos(KClat)) * (np.sin(chlon/2)**2))
np.sin((chlat/2.))
np.cos(lat)

In [ ]:
KC_Metro_comparison_sorted = KC_Metro_comparison.sort_values("Distance (miles)")
KC_Metro_comparison_sorted

In [ ]:
#Plot distance from downtown vs change in temperature
KC_Metro_comparison_sorted.plot.bar(x="Distance (miles)", y=["Day1 Chng","Day2 Chng", "Day3 Chng", "Day4 Chng", "Day5 Chng"], figsize=(20,10))
#KC_Metro_comparison_sorted.plot.bar(x="Distance (miles)", y="Day2 Chng", figsize=(20,3))
plt.title("Change in Heat with Respect to Distance from Downtown KC")
plt.xlabel("Distance (miles)")
plt.ylabel("Change in Temperature")

In [ ]:
#Linear Regression Analysis
#make distance v change in temp data set
D1_Temp_V_Dist = KC_Metro_comparison[["Distance (miles)", "Day1 Chng"]]
D2_Temp_V_Dist = KC_Metro_comparison[["Distance (miles)", "Day2 Chng"]]
D3_Temp_V_Dist = KC_Metro_comparison[["Distance (miles)", "Day3 Chng"]]
D4_Temp_V_Dist = KC_Metro_comparison[["Distance (miles)", "Day4 Chng"]]
D5_Temp_V_Dist = KC_Metro_comparison[["Distance (miles)", "Day5 Chng"]]

#Rename new dataframe columns to prepare for combining dataframes
D1_Temp_V_Dist = D1_Temp_V_Dist.rename(columns={"Day1 Chng": "Temp Change"})
D2_Temp_V_Dist = D1_Temp_V_Dist.rename(columns={"Day2 Chng": "Temp Change"})
D3_Temp_V_Dist = D1_Temp_V_Dist.rename(columns={"Day3 Chng": "Temp Change"})
D4_Temp_V_Dist = D1_Temp_V_Dist.rename(columns={"Day4 Chng": "Temp Change"})
D5_Temp_V_Dist = D1_Temp_V_Dist.rename(columns={"Day5 Chng": "Temp Change"})

#Concatinate dataframes
#df.append(df2)
#pd.concat([t1, t2, t3, t4, t5])
Temp_V_Dist = D1_Temp_V_Dist.append(D2_Temp_V_Dist)
Temp_V_Dist = Temp_V_Dist.append(D3_Temp_V_Dist)
Temp_V_Dist = Temp_V_Dist.append(D4_Temp_V_Dist)
Temp_V_Dist = Temp_V_Dist.append(D5_Temp_V_Dist)
Temp_V_Dist

In [ ]:
ax = KC_Metro_comparison.plot.scatter(x="Distance (miles)", y="Day1 Chng", color="DarkBlue", label="Day 1", figsize=(10,7))
ax1 = KC_Metro_comparison.plot.scatter(x="Distance (miles)", y="Day2 Chng", color="DarkGreen", label="Day 2", ax=ax)
ax2 = KC_Metro_comparison.plot.scatter(x="Distance (miles)", y="Day3 Chng", color="Purple", label="Day 3", ax=ax)
ax3 = KC_Metro_comparison.plot.scatter(x="Distance (miles)", y="Day4 Chng", color="DarkRed", label="Day 4", ax=ax)
ax4 = KC_Metro_comparison.plot.scatter(x="Distance (miles)", y="Day5 Chng", color="DarkCyan", label="Day 5", ax=ax)

x_values = Temp_V_Dist["Distance (miles)"]
y_values = Temp_V_Dist["Temp Change"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(5, 1),fontsize=25,color="red")
print(f"The r-squared is: {round(rvalue**2, 2)}")
plt.title("Change in High Temperature with Respect to Distance from Downtown KC")
plt.xlabel("Distance (miles)")
plt.xticks(rotation="vertical")
plt.xlim(-1, max(KC_Metro_comparison["Distance (miles)"])+1)
plt.ylim(-2.6, 2.5)
plt.tight_layout()


In [ ]:
ax = KC_Metro_comparison.plot.scatter(x="Distance (miles)", y="Day1 Chng", color="DarkBlue", label="Day 1", figsize=(10,7))
ax1 = KC_Metro_comparison.plot.scatter(x="Distance (miles)", y="Day2 Chng", color="DarkGreen", label="Day 2", ax=ax)
ax2 = KC_Metro_comparison.plot.scatter(x="Distance (miles)", y="Day3 Chng", color="Purple", label="Day 3", ax=ax)
ax3 = KC_Metro_comparison.plot.scatter(x="Distance (miles)", y="Day4 Chng", color="DarkRed", label="Day 4", ax=ax)
ax4 = KC_Metro_comparison.plot.scatter(x="Distance (miles)", y="Day5 Chng", color="DarkCyan", label="Day 5", ax=ax)

x_values = KC_Metro_comparison["Distance (miles)"]
y_values = KC_Metro_comparison["Day1 Chng"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,regress_values,"r-",color="DarkBlue")
plt.annotate(line_eq,(5, 2),fontsize=25,color="DarkBlue")
print(f"The r-squared is: {round(rvalue**2, 2)}")

x_values = KC_Metro_comparison["Distance (miles)"]
y2_values = KC_Metro_comparison["Day2 Chng"]
(slope2, intercept2, rvalue2, pvalue2, stderr2) = linregress(x_values, y2_values)
regress_values2 = x_values * slope2 + intercept2
line_eq2 = "y = " + str(round(slope2,2)) + "x + " + str(round(intercept2,2))
plt.plot(x_values,regress_values2,"r-",color="DarkGreen")
plt.annotate(line_eq2,(5, 1),fontsize=25,color="DarkGreen")
print(f"The r-squared is: {round(rvalue2**2, 2)}")

x_values = KC_Metro_comparison["Distance (miles)"]
y3_values = KC_Metro_comparison["Day3 Chng"]
(slope3, intercept3, rvalue3, pvalue3, stderr3) = linregress(x_values, y3_values)
regress_values3 = x_values * slope3 + intercept3
line_eq3 = "y = " + str(round(slope3,2)) + "x + " + str(round(intercept3,2))
plt.plot(x_values,regress_values3,"r-",color="Purple")
plt.annotate(line_eq3,(5, 0),fontsize=25,color="Purple")
print(f"The r-squared is: {round(rvalue3**2, 2)}")

x_values = KC_Metro_comparison["Distance (miles)"]
y4_values = KC_Metro_comparison["Day4 Chng"]
(slope4, intercept4, rvalue4, pvalue4, stderr4) = linregress(x_values, y4_values)
regress_values4 = x_values * slope4 + intercept4
line_eq4 = "y = " + str(round(slope4,2)) + "x + " + str(round(intercept4,2))
plt.plot(x_values,regress_values4,"r-",color="DarkRed")
plt.annotate(line_eq4,(5, -1),fontsize=25,color="DarkRed")
print(f"The r-squared is: {round(rvalue4**2, 2)}")

x_values = KC_Metro_comparison["Distance (miles)"]
y5_values = KC_Metro_comparison["Day5 Chng"]
(slope5, intercept5, rvalue5, pvalue5, stderr5) = linregress(x_values, y5_values)
regress_values5 = x_values * slope5 + intercept5
line_eq5 = "y = " + str(round(slope5,-2)) + "x + " + str(round(intercept5,2))
plt.plot(x_values,regress_values5,"r-",color="DarkCyan")
plt.annotate(line_eq5,(5, -2),fontsize=25,color="DarkCyan")
print(f"The r-squared is: {round(rvalue5**2, 2)}")

plt.title("Change in High Temperature with Respect to Distance from Downtown KC")
plt.xlabel("Distance (miles)")
plt.xticks(rotation="vertical")
plt.xlim(-1, max(KC_Metro_comparison["Distance (miles)"])+1)
plt.ylim(-2.6, 2.5)
plt.tight_layout()
plt.savefig("temp_vs_dist.png")

In [ ]:
KC_Metro_comparison.describe()

# Collect Population Data from Census API

In [ ]:
census_data = c.acs5.get(("NAME", "B01003_001E"), 
                         {'for': 'zip code tabulation area:*'})
#, 'in': 'state:{}'.format(states.KS.fips)
                     #       {'for': 'zip code tabulation area:*'}

#for=zip+code+tabulation+area:*&in=state:02
#c.acs5.get(('NAME', 'B25034_010E'),
#          {'for': 'state:{}'.format(states.MD.fips)})
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

census_pd = census_pd[["Zipcode", "Population"]]
                       
census_pd

In [ ]:
#Read josn file listing all zip codes in KS
#json file was downloaded from https://public.opendatasoft.com/
Zips_All = pd.read_json("ProjectOne/Zip_Lat_Lon.json", orient="list")
Zips_All['geometry']

In [ ]:
Zips_All.columns

In [ ]:
Zips_All['fields']

In [ ]:
Zip = pd.concat([Zips_All.drop(['fields'], axis=1), Zips_All['fields'].apply(pd.Series)], axis=1)
Zip_Codes = Zip[["city", "state", "zip", "latitude", "longitude"]]
Zip_Codes = Zip_Codes.rename(columns={"zip": "Zipcode"})
Zip_Codes

In [ ]:
Population_df = pd.merge(census_pd, Zip_Codes, on="Zipcode")

In [ ]:
Population_df.dtypes

In [ ]:
Pop_coord = []
Pop_markers = []

for index, row in Population_df.iterrows():
    lat = row["latitude"]
    lon = row["longitude"]
    Pop_coord.append((lat, lon))


#for index, row in KC_Metro.iterrows():
    population = row["Population"]
    Pop_markers.append(str(population))
Pop_coord

In [ ]:
KS_city_data

In [ ]:
reduced_coords = []
reduced_temps = []

for index, row in KS_city_data.iterrows():
    if index % 5 == 0:
        lat = row["Lat"]
        lon = row["Lng"]
        reduced_coords.append((lat, lon))
    
        temp = row["Day1 High"]
        reduced_temps.append(str(temp))
    
reduced_coords

In [ ]:
# Map temperature in all cities in list
fig = gmaps.figure(center=(38.5,-98.35), zoom_level=7)
#fig = gmaps.figure(map_type='HYBRID')
heat_layer = gmaps.heatmap_layer(Pop_coord, weights=Population_df["Population"], 
                                 dissipating=False, max_intensity=Population_df["Population"].max(),
                                 point_radius = 0.1)

marker_layer = gmaps.marker_layer(reduced_coords, label=reduced_temps)

# Add the layer to the map
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig